In [1]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, PointDrawTool, Div, Range1d, Ellipse
from bokeh.layouts import column
from bokeh.layouts import gridplot
from bokeh.io import show
from bokeh.models.callbacks import CustomJS
import numpy as np

In [2]:
def ellipse_point(cx, cy, a, b, theta, t):
    c, s = np.cos(theta), np.sin(theta)
    x = cx + a*np.cos(t)*c - b*np.sin(t)*s
    y = cy + a*np.cos(t)*s + b*np.sin(t)*c
    return x, y

In [3]:
def C2toBloch(re0, im0, re1, im1, v_stretch=0.3, eps=1e-12):
    # alpha0 and alpha1 are the complex amplitudes
    alpha0 = re0 + 1j*im0
    alpha1 = re1 + 1j*im1

    # normalize (if all-zero, return +Z pole)
    a0mod = abs(alpha0)**2
    a1mod = abs(alpha1)**2
    n2 = (a0mod + a1mod)
    if n2 < eps:
        return 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0

    alpha0 /= np.sqrt(n2)
    alpha1 /= np.sqrt(n2)

    # Bloch components for a pure state:
    # x = 2 Re(α β*), y = 2 Im(α β*), z = |α|^2 - |β|^2
    ab_conj = alpha0 * np.conj(alpha1)
    x = 2.0 * np.real(ab_conj)
    y = 2.0 * np.imag(ab_conj)
    z = float(abs(alpha0)**2 - abs(alpha1)**2)

    # angles in spherical coordinates
    z_clamped = np.clip(z, -1.0, 1.0)
    theta = float(np.arccos(z_clamped))     # z = cos θ, θ ∈ [0, π]
    phi   = float(np.arctan2(y, x))         # φ = atan2(y, x), φ ∈ [-π, π]
    
    # 3D Bloch sphere coordinates
    # x_3d = sin(θ)cos(φ)  -- points "forward/back" (compressed in 2D view)
    # y_3d = sin(θ)sin(φ)  -- points "left/right" (horizontal in 2D view)
    # z_3d = cos(θ)         -- points "up/down" (vertical in 2D view)
    
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    cos_phi = np.cos(phi)
    sin_phi = np.sin(phi)
    
    x_3d = sin_theta * cos_phi
    y_3d = sin_theta * sin_phi
    z_3d = cos_theta
    
    # 2D projection (viewing the sphere from a tilted angle)
    # Horizontal position: y-component (left-right)
    # Vertical position: z-component + foreshortened x-component
    x_2d = float(y_3d)
    y_2d = float(z_3d + v_stretch * x_3d)
    
    # For the ellipse at this z-height:
    # The ellipse center is at (0, z_3d)
    # The radius in 3D is sin(θ)
    # In 2D: horizontal radius = sin(θ), vertical radius = v_stretch * sin(θ)
    ellipse_radius = float(sin_theta)

    return theta, phi, float(x), float(y), float(z), float(x_2d), float(y_2d), ellipse_radius

In [4]:
lim = 1.15

# data sources: first probabilistic amplitude is set to 1 + 0j (modulus and line manually entered)
a0re = 1.0;
a0im = 0.0;
pt0m = np.sqrt(a0re**2 + a0im**2)
pt0 = ColumnDataSource(dict(x=[a0re], y=[a0im]))
line0 = ColumnDataSource(dict(x=[0.0, a0re], y = [0.0, a0im]))

# data sources: second probabilistic amplitude is set to 0. (modulus and line manually entered)
a1re = 0.0;
a1im = 0.0;
pt1m = np.sqrt(a1re**2 + a1im**2)
pt1 = ColumnDataSource(dict(x=[a1re], y=[a1im]))
line1 = ColumnDataSource(dict(x=[0.0, a1re], y =[0.0, a1im]))

# circle sources 
thetacircle = np.linspace(0, 2*np.pi, 200)
xcircle = np.cos(thetacircle)
ycircle = np.sin(thetacircle)
# JS needs plain lists
ux = xcircle.tolist()  
uy = ycircle.tolist()
circ0_src = ColumnDataSource(dict(x=(pt0m*xcircle).tolist(), y=(pt0m*ycircle).tolist()))
circ1_src = ColumnDataSource(dict(x=(pt1m*xcircle).tolist(), y=(pt1m*ycircle).tolist()))

# ellipse sources: params (center xc, yc; semi-axes w, h; angle ang in radians)

# prepare ellipse to convey 3d-ness of Bloch sphere
v_stretch = 0.3
ellipse_cosy_src = ColumnDataSource(data=dict(xc=[0.0], yc=[0.0], w=[2*1.0], h=[2*v_stretch], ang=[0.]))

# Reference axis: draw the x-axis (φ=0 direction) which points "forward" in 3D
# At equator (θ=π/2), φ=0 means x=1, y=0, z=0 in 3D Bloch coords
# In 2D projection: x_2d = 0 (since y_3d=0), y_2d = 0 + v_stretch*1 = v_stretch
# At opposite side (φ=π): x_2d = 0, y_2d = 0 + v_stretch*(-1) = -v_stretch
ellipse_diam_cosy_src = ColumnDataSource(data=dict(x=[0.0, 0.0], y=[v_stretch, -v_stretch]))

# prepare ellipse to position qubit of varying relative phase (uses Bloch transformation)
b_theta, b_phi, b_x, b_y, b_z, b_x2d, b_y2d, b_ellipse_radius = C2toBloch(
    pt0.data['x'][0], pt0.data['y'][0], 
    pt1.data['x'][0], pt1.data['y'][0],
    v_stretch=v_stretch
)

# data source for qubit position on Bloch sphere
qubit = ColumnDataSource(dict(x=[b_x2d], y=[b_y2d]))

# ellipse at the z-height of the qubit
ellipse_qubit_src = ColumnDataSource(data=dict(
    xc=[0.0], 
    yc=[b_z],  # z-coordinate (vertical position of ellipse center)
    w=[2*b_ellipse_radius],  # horizontal diameter
    h=[2*b_ellipse_radius*v_stretch],  # vertical diameter (foreshortened)
    ang=[0.]
))

# complex plane
p = figure(width=420, height=420,
           x_range=Range1d(-lim, lim), y_range=Range1d(-lim, lim),
           match_aspect=True, toolbar_location="above",  tools="pan,wheel_zoom,reset" , title="Complex Plane")
p.toolbar.logo = None
p.xaxis.axis_label = "Re"
p.yaxis.axis_label = "Im"
p.grid.visible = True

# draw unit circle for reference
p.line(xcircle, ycircle, line_width=2, color="navy")

# draw admissibility circle of a0 and a1
circ0 = p.line('x', 'y', source=circ0_src, line_width=1, color="crimson", alpha = 0.2)
circ1 = p.line('x', 'y', source=circ1_src, line_width=1, color="green", alpha = 0.2)

# draw point and line for alpha_0
seg0 = p.line('x', 'y', source=line0, line_width=3)
r0 = p.scatter('x', 'y', source=pt0, size=12, color="crimson")

# draw point and line for alpha_1
seg1 = p.line('x', 'y', source=line1, line_width=3)
r1 = p.scatter('x', 'y', source=pt1, size=12, color="green")

# enable dragging of the single point (no adding/removing)
drag = PointDrawTool(renderers=[r0,r1], add=False)
p.add_tools(drag)
p.toolbar.active_drag = drag

# live readout
readout0 = Div(text="Re(a0) = 1.000 &nbsp;&nbsp; Im(a0) = 0.000")#, style={"font-family": "monospace"})
readout1 = Div(text="Re(a1) = 0.000 &nbsp;&nbsp; Im(a1) = 0.000")#, style={"font-family": "monospace"})

b = figure(width=420, height=420,
           x_range=Range1d(-lim, lim), y_range=Range1d(-lim, lim),
           match_aspect=True, toolbar_location="above",  tools="pan,wheel_zoom,reset" , title="Bloch sphere")
b.toolbar.logo = None
b.xaxis.axis_label = "Re"
b.yaxis.axis_label = "Im"
b.grid.visible = True

# draw unit circle for reference
b.line(xcircle, ycircle, line_width=2, color="navy")
                                               
# draw ellipse to give an indication of a second dimension
b.add_glyph(ellipse_cosy_src, Ellipse(
    x='xc', y='yc', width='w', height='h', angle='ang',
    fill_alpha=0.08, line_width=2
))

# draw horizontal line for reference (y-axis)
b.line(xcircle, 0.0*ycircle, line_width=0.75, color="black", line_alpha=0.9)

# draw vertical line for reference (z-axis)
b.line(0.0*xcircle, ycircle, line_width=0.75, color="black", line_alpha=0.9)

# draw front-pointing line for reference (x-axis)
b.line('x', 'y', source=ellipse_diam_cosy_src, line_width=0.75, color="black", line_alpha=0.9)

# draw point and ellipse for the qubit into the Bloch sphere
qbitp = b.scatter('x', 'y', source=qubit, size=12, color="black")
qubit_ellipse = b.add_glyph(ellipse_qubit_src, Ellipse(
    x='xc', y='yc', width='w', height='h', angle='ang',
    fill_alpha=0.08, line_width=0.5
))

# grid with a single merged toolbar
grid = gridplot(
    [[b, p]],
    toolbar_location="above",
    merge_tools=True          # merge pan/zoom/reset (only one toolbar shown)
)

# JS callback updates the lines, circles and readout when the points moves
pt0.js_on_change('data', CustomJS(args=dict(s0=pt0, s1=pt1, sq = qubit, eq = ellipse_qubit_src, l0=line0, l1 = line1, c0=circ0_src, c1=circ1_src, ux=ux, uy=uy, d0=readout0, d1=readout1, v_stretch=v_stretch), code="""
  
  // Shared lock across callbacks
  const L = window.__blochLock__ || (window.__blochLock__ = { s0:false, s1:false });

  // if this handler is muted, bail out
  if (L.s0) return;
  L.s0 = true;
  
  try {
      // drag new point
      const re = s0.data.x[0] ?? 0;
      const im = s0.data.y[0] ?? 0;

      // redraw line
      l0.data.x = [0, re];
      l0.data.y = [0, im];
      l0.change.emit();

      // redraw admissibility circle a0
      const r0 = Math.hypot(re, im);
      c0.data.x = ux.map(v => r0 * v);
      c0.data.y = uy.map(v => r0 * v);
      c0.change.emit();

      // compute what's needed to adjust s1, l1, and c1
      const r1 = r0*r0 <= 1 ? Math.sqrt(1 - r0*r0) : 0;
      const gamma1 = (s1.data.x[0] === 0 && s1.data.y[0] === 0) ? 0 : Math.atan2(s1.data.y[0], s1.data.x[0]);
      const re1 = r1 * Math.cos(gamma1);
      const im1 = r1 * Math.sin(gamma1);

      try {
          // mute pt1's handler while s1 is updated
          L.s1 = true;
          s1.data.x[0] = re1;
          s1.data.y[0] = im1;
          s1.change.emit();
          // IMPORTANT: clear selection so PointDrawTool won't grab s1 next time
          s1.selected.indices = [];
          s1.selected.change.emit?.(); 
          } finally {
          L.s1 = false;
        }
      
      l1.data.x = [0, re1];
      l1.data.y = [0, im1];
      l1.change.emit();

      // redraw admissibility circle a1
      c1.data.x = ux.map(v => r1 * v);
      c1.data.y = uy.map(v => r1 * v);
      c1.change.emit();
      
      // Compute Bloch sphere coordinates using proper 3D->2D projection
      // Normalize the amplitudes
      const a0mod = re*re + im*im;
      const a1mod = re1*re1 + im1*im1;
      const norm = Math.sqrt(a0mod + a1mod);
      
      if (norm < 1e-12) {
          // All-zero case: point at +Z pole
          sq.data.x[0] = 0.0;
          sq.data.y[0] = 1.0;
          eq.data.yc[0] = 1.0;
          eq.data.w[0] = 0.0;
          eq.data.h[0] = 0.0;
      } else {
          const alpha0_re = re / norm;
          const alpha0_im = im / norm;
          const alpha1_re = re1 / norm;
          const alpha1_im = im1 / norm;
          
          // Bloch vector components
          // ab_conj = alpha0 * conj(alpha1)
          const ab_re = alpha0_re * alpha1_re + alpha0_im * alpha1_im;
          const ab_im = alpha0_im * alpha1_re - alpha0_re * alpha1_im;
          
          const x_bloch = 2.0 * ab_re;
          const y_bloch = 2.0 * ab_im;
          const z_bloch = (alpha0_re*alpha0_re + alpha0_im*alpha0_im) - 
                          (alpha1_re*alpha1_re + alpha1_im*alpha1_im);
          
          // Spherical angles
          const z_clamped = Math.max(-1.0, Math.min(1.0, z_bloch));
          const theta = Math.acos(z_clamped);
          const phi = Math.atan2(y_bloch, x_bloch);
          
          // 3D coordinates on Bloch sphere
          const sin_theta = Math.sin(theta);
          const cos_theta = Math.cos(theta);
          const cos_phi = Math.cos(phi);
          const sin_phi = Math.sin(phi);
          
          const x_3d = sin_theta * cos_phi;  // forward/back
          const y_3d = sin_theta * sin_phi;  // left/right
          const z_3d = cos_theta;            // up/down
          
          // 2D projection
          const x_2d = y_3d;
          const y_2d = z_3d + v_stretch * x_3d;
          
          // Update qubit position
          sq.data.x[0] = x_2d;
          sq.data.y[0] = y_2d;
          
          // Update ellipse at this z-height
          const ellipse_radius = sin_theta;
          eq.data.xc[0] = 0.0;
          eq.data.yc[0] = z_3d;
          eq.data.w[0] = 2.0 * ellipse_radius;
          eq.data.h[0] = 2.0 * ellipse_radius * v_stretch;
      }
      
      eq.change.emit();
      sq.change.emit();

      // add text
      d0.text = `Re(a0) = ${re.toFixed(3)} &nbsp;&nbsp; Im(a0) = ${im.toFixed(3)}`;
      d1.text = `Re(a1) = ${re1.toFixed(3)} &nbsp;&nbsp; Im(a1) = ${im1.toFixed(3)}`;

      } finally {
        // ALWAYS release, even if an error occurs
        L.s0 = false;}
    """ ))

layout = column(grid, readout0, readout1
    #sizing_mode="stretch_width"  
)

# this is to open in a browser immediately
show(layout)

# this is to save as stand-alone html
output_file("bloch_sphere.html", title="Interactive Bloch Sphere Generator", mode="inline")
save(layout)

/tmp/ipykernel_12263/1815321565.py:125: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  grid = gridplot(


'/home/sathya/Desktop/Interactive_Bloch/bloch_sphere.html'

Gtk-Message: 13:56:30.462: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
